In [431]:
import pandas as pd
import numpy as np
import json
from datetime import datetime
from dotenv import load_dotenv
import os
import requests


In [432]:
df = pd.read_csv('tmdb_5000_movies.csv')
df.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [433]:
df.shape

(4803, 20)

In [434]:
df.status.value_counts()

Released           4795
Rumored               5
Post Production       3
Name: status, dtype: int64

In [435]:
df = df[df.status == 'Released' ]

In [436]:
df.drop(['homepage','original_title', 'status','tagline','spoken_languages','overview'], axis=1, inplace=True)

In [437]:
cols = df.columns.tolist()
cols

['budget',
 'genres',
 'id',
 'keywords',
 'original_language',
 'popularity',
 'production_companies',
 'production_countries',
 'release_date',
 'revenue',
 'runtime',
 'title',
 'vote_average',
 'vote_count']

In [438]:
cols2 = ['id','title','release_date','runtime','original_language','genres','production_companies',
        'production_countries','keywords','budget','vote_average','vote_count','popularity','revenue']

In [439]:
df = df[cols2]
df.head()

,id,title,release_date,runtime,original_language,genres,production_companies,production_countries,keywords,budget,vote_average,vote_count,popularity,revenue
0,19995,Avatar,2009-12-10,162.0,en,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",237000000,7.2,11800,150.437577,2787965087
1,285,Pirates of the Caribbean: At World's End,2007-05-19,169.0,en,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",300000000,6.9,4500,139.082615,961000000
2,206647,Spectre,2015-10-26,148.0,en,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",245000000,6.3,4466,107.376788,880674609
3,49026,The Dark Knight Rises,2012-07-16,165.0,en,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",250000000,7.6,9106,112.312950,1084939099
4,49529,John Carter,2012-03-07,132.0,en,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",260000000,6.1,2124,43.926995,284139100


In [440]:
df.original_language.value_counts()[:4]

en    4498
fr      70
es      32
de      27
Name: original_language, dtype: int64

In [441]:
df.original_language.value_counts()

en    4498
fr      70
es      32
de      27
zh      27
hi      19
ja      16
it      14
cn      12
ko      11
ru      11
pt       9
da       7
sv       5
fa       4
nl       4
he       3
th       3
id       2
ro       2
ta       2
ar       2
vi       1
is       1
pl       1
no       1
af       1
sl       1
ps       1
tr       1
el       1
hu       1
te       1
ky       1
nb       1
cs       1
xx       1
Name: original_language, dtype: int64

In [442]:
def convert_lang(elem):
    top_countries = df.original_language.value_counts()[:4]
    if elem not in top_countries:
        elem = 'other'
    return elem            

In [443]:
df.original_language = df.original_language.apply(convert_lang)
df.head()

,id,title,release_date,runtime,original_language,genres,production_companies,production_countries,keywords,budget,vote_average,vote_count,popularity,revenue
0,19995,Avatar,2009-12-10,162.0,en,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",237000000,7.2,11800,150.437577,2787965087
1,285,Pirates of the Caribbean: At World's End,2007-05-19,169.0,en,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",300000000,6.9,4500,139.082615,961000000
2,206647,Spectre,2015-10-26,148.0,en,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",245000000,6.3,4466,107.376788,880674609
3,49026,The Dark Knight Rises,2012-07-16,165.0,en,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",250000000,7.6,9106,112.312950,1084939099
4,49529,John Carter,2012-03-07,132.0,en,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",260000000,6.1,2124,43.926995,284139100


In [444]:
df.original_language.value_counts()

en       4498
other     168
fr         70
es         32
de         27
Name: original_language, dtype: int64

In [445]:
df[df.vote_count==0].vote_count.value_counts()

0    61
Name: vote_count, dtype: int64

In [446]:
df[df.popularity==0].popularity.value_counts()

0.0    1
Name: popularity, dtype: int64

In [447]:
df = df[df.vote_count!=0]

In [448]:
df = df[df.popularity!=0]

In [449]:
df.shape

(4734, 14)

In [450]:
df = df[df.id!=112430]

In [451]:
df.shape

(4733, 14)

In [452]:
df = df[(df.runtime>80) | ((df.budget!=0) & (df.revenue!=0))]

In [453]:
df.shape

(4645, 14)

In [454]:
df2 = df.copy()

In [455]:
# all_genres = []
# for x in df.genres:
#     x = json.loads(x)
#     x = [z['name'] for z in x]
#     for y in x:
#         if y not in all_genres:
#             all_genres.append(y)
# all_genres

In [456]:
def getNames(series):
    lis = []
    for x in series:
        x = json.loads(x)
        x = [z['name'] for z in x]
        for y in x:
            if y not in lis:
                lis.append(y)
    return lis

all_genres = getNames(df.genres)
all_genres

['Action',
 'Adventure',
 'Fantasy',
 'Science Fiction',
 'Crime',
 'Drama',
 'Thriller',
 'Animation',
 'Family',
 'Western',
 'Comedy',
 'Romance',
 'Horror',
 'Mystery',
 'History',
 'War',
 'Music',
 'Documentary',
 'Foreign',
 'TV Movie']

In [457]:
def convert_genres(elem):
    elem = json.loads(elem)
    elem = [z['name'] for z in elem]
    return elem

# def convert_genres2(elem):
#     elem = json.loads(elem)
#     res = ''
#     for z in elem:
#         res += z['name']
#     return res

# def convert_genres3(elem):
#     elem = json.loads(elem)
#     elem = [z['name'] for z in elem]
#     return set(elem)

In [458]:
df.genres = df.genres.apply(convert_genres)
df.head()

,id,title,release_date,runtime,original_language,genres,production_companies,production_countries,keywords,budget,vote_average,vote_count,popularity,revenue
0,19995,Avatar,2009-12-10,162.0,en,"[Action, Adventure, Fantasy, Science Fiction]","[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",237000000,7.2,11800,150.437577,2787965087
1,285,Pirates of the Caribbean: At World's End,2007-05-19,169.0,en,"[Adventure, Fantasy, Action]","[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",300000000,6.9,4500,139.082615,961000000
2,206647,Spectre,2015-10-26,148.0,en,"[Action, Adventure, Crime]","[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",245000000,6.3,4466,107.376788,880674609
3,49026,The Dark Knight Rises,2012-07-16,165.0,en,"[Action, Crime, Drama, Thriller]","[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",250000000,7.6,9106,112.312950,1084939099
4,49529,John Carter,2012-03-07,132.0,en,"[Action, Adventure, Science Fiction]","[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",260000000,6.1,2124,43.926995,284139100


In [459]:
genres_dummies = df['genres'].str.join(sep='*').str.get_dummies(sep='*')
genres_dummies.head()

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,Foreign,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
4,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [460]:
all_comps = []
for x in df.production_companies:
    if len(x)>2:
        x = json.loads(x)
        y = x[0]['name']
    
        if y not in all_comps:
                all_comps.append(y)
len(all_comps)

1257

In [461]:
df.drop('production_companies', axis=1, inplace=True)

In [462]:
all_countries = getNames(df.production_countries)
display(len(all_countries), all_countries[:10])

83

['United States of America',
 'United Kingdom',
 'Jamaica',
 'Bahamas',
 'Dominica',
 'Czech Republic',
 'Poland',
 'Slovenia',
 'New Zealand',
 'Germany']

In [463]:
df.production_countries = df.production_countries.apply(convert_genres)
df.head()

,id,title,release_date,runtime,original_language,genres,production_countries,keywords,budget,vote_average,vote_count,popularity,revenue
0,19995,Avatar,2009-12-10,162.0,en,"[Action, Adventure, Fantasy, Science Fiction]","[United States of America, United Kingdom]","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",237000000,7.2,11800,150.437577,2787965087
1,285,Pirates of the Caribbean: At World's End,2007-05-19,169.0,en,"[Adventure, Fantasy, Action]",[United States of America],"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",300000000,6.9,4500,139.082615,961000000
2,206647,Spectre,2015-10-26,148.0,en,"[Action, Adventure, Crime]","[United Kingdom, United States of America]","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",245000000,6.3,4466,107.376788,880674609
3,49026,The Dark Knight Rises,2012-07-16,165.0,en,"[Action, Crime, Drama, Thriller]",[United States of America],"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",250000000,7.6,9106,112.312950,1084939099
4,49529,John Carter,2012-03-07,132.0,en,"[Action, Adventure, Science Fiction]",[United States of America],"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",260000000,6.1,2124,43.926995,284139100


In [464]:
countries_dummies = df['production_countries'].str.join(sep='*').str.get_dummies(sep='*')
countries_dummies.head()

,Afghanistan,Angola,Argentina,Aruba,Australia,Austria,Bahamas,Belgium,Bhutan,Bolivia,...,Sweden,Switzerland,Taiwan,Thailand,Tunisia,Turkey,Ukraine,United Arab Emirates,United Kingdom,United States of America
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [465]:
all_keys = getNames(df.keywords)
display(len(all_keys), all_keys[:10])

9765

['culture clash',
 'future',
 'space war',
 'space colony',
 'society',
 'space travel',
 'futuristic',
 'romance',
 'space',
 'alien']

In [466]:
df.drop('keywords', axis=1, inplace=True)

In [467]:
df['release_date'] = pd.to_datetime(df['release_date'])
df.dtypes

id                               int64
title                           object
release_date            datetime64[ns]
runtime                        float64
original_language               object
genres                          object
production_countries            object
budget                           int64
vote_average                   float64
vote_count                       int64
popularity                     float64
revenue                          int64
dtype: object

In [468]:
df.head()

,id,title,release_date,runtime,original_language,genres,production_countries,budget,vote_average,vote_count,popularity,revenue
0,19995,Avatar,2009-12-10,162.0,en,"[Action, Adventure, Fantasy, Science Fiction]","[United States of America, United Kingdom]",237000000,7.2,11800,150.437577,2787965087
1,285,Pirates of the Caribbean: At World's End,2007-05-19,169.0,en,"[Adventure, Fantasy, Action]",[United States of America],300000000,6.9,4500,139.082615,961000000
2,206647,Spectre,2015-10-26,148.0,en,"[Action, Adventure, Crime]","[United Kingdom, United States of America]",245000000,6.3,4466,107.376788,880674609
3,49026,The Dark Knight Rises,2012-07-16,165.0,en,"[Action, Crime, Drama, Thriller]",[United States of America],250000000,7.6,9106,112.312950,1084939099
4,49529,John Carter,2012-03-07,132.0,en,"[Action, Adventure, Science Fiction]",[United States of America],260000000,6.1,2124,43.926995,284139100


In [469]:
df.shape

(4645, 12)

In [470]:
df[df.revenue==0].revenue.value_counts()

0    1279
Name: revenue, dtype: int64

In [471]:
df[df.revenue==0]

,id,title,release_date,runtime,original_language,genres,production_countries,budget,vote_average,vote_count,popularity,revenue
83,79698,The Lovers,2015-02-13,109.0,en,"[Action, Adventure, Science Fiction, Romance]","[Australia, Belgium, India]",27000000,4.8,34,2.418535,0
135,7978,The Wolfman,2010-02-11,102.0,en,"[Drama, Horror, Thriller]",[United States of America],150000000,5.5,549,21.214571,0
265,10588,The Cat in the Hat,2003-11-21,82.0,en,"[Comedy, Fantasy, Family]",[United States of America],0,4.9,366,18.251129,0
309,10214,Son of the Mask,2005-02-18,94.0,en,"[Fantasy, Comedy, Family, Adventure]","[Germany, United States of America]",84000000,3.6,338,17.815595,0
376,10357,Volcano,1997-04-25,104.0,en,"[Science Fiction, Action, Drama, Thriller]",[United States of America],90000000,5.2,376,19.836124,0
406,51052,Arthur Christmas,2011-02-22,97.0,en,"[Drama, Animation, Family, Comedy]",[United States of America],0,6.7,333,19.836830,0
433,146216,RED 2,2013-07-18,116.0,en,"[Action, Comedy, Crime, Thriller]","[Canada, France, United States of America]",84000000,6.4,1526,44.343330,0
453,12096,The Pink Panther,2006-01-18,93.0,en,"[Action, Comedy, Crime, Mystery, Family]",[United States of America],0,5.6,550,25.450534,0
463,161795,Déjà Vu,1998-04-22,117.0,en,"[Romance, Drama]",[United States of America],0,8.0,1,0.605645,0
467,9822,Mighty Joe Young,1998-12-25,114.0,en,"[Action, Adventure, Family, Fantasy]",[United States of America],90000000,5.9,208,6.643778,0


In [472]:
df[df.budget==0].budget.value_counts()

0    915
Name: budget, dtype: int64

In [473]:
load_dotenv(dotenv_path='.env')
SECRET_KEY = os.getenv("TOKEN")

In [483]:
id1 = str(19995)
response = requests.get('https://api.themoviedb.org/3/movie/'+id1+'/external_ids?api_key='+SECRET_KEY)
results = response.json()
display('imdb_id' in results.keys())

True

In [485]:
def imdb_id(row):
    if row['budget']==0 or row['revenue']==0:
        id1 = str(row['id'])
        response = requests.get('https://api.themoviedb.org/3/movie/'+id1+'/external_ids?api_key='+SECRET_KEY)
        results = response.json()
        if 'imdb_id' in results.keys():
            return results["imdb_id"]
        else: return 0
    else: return 0

df['imdb_id'] = df.apply(imdb_id, axis=1)

In [490]:
df[df.revenue==0].head()

,id,title,release_date,runtime,original_language,genres,production_countries,budget,vote_average,vote_count,popularity,revenue,imdb_id
83,79698,The Lovers,2015-02-13,109.0,en,"[Action, Adventure, Science Fiction, Romance]","[Australia, Belgium, India]",27000000,4.8,34,2.418535,0,tt1321869
135,7978,The Wolfman,2010-02-11,102.0,en,"[Drama, Horror, Thriller]",[United States of America],150000000,5.5,549,21.214571,0,tt0780653
265,10588,The Cat in the Hat,2003-11-21,82.0,en,"[Comedy, Fantasy, Family]",[United States of America],0,4.9,366,18.251129,0,tt0312528
309,10214,Son of the Mask,2005-02-18,94.0,en,"[Fantasy, Comedy, Family, Adventure]","[Germany, United States of America]",84000000,3.6,338,17.815595,0,tt0362165
376,10357,Volcano,1997-04-25,104.0,en,"[Science Fiction, Action, Drama, Thriller]",[United States of America],90000000,5.2,376,19.836124,0,tt0120461


In [514]:
df.head()

,id,title,release_date,runtime,original_language,genres,production_countries,budget,vote_average,vote_count,popularity,revenue,imdb_id
0,19995,Avatar,2009-12-10,162.0,en,"[Action, Adventure, Fantasy, Science Fiction]","[United States of America, United Kingdom]",237000000,7.2,11800,150.437577,2787965087,0
1,285,Pirates of the Caribbean: At World's End,2007-05-19,169.0,en,"[Adventure, Fantasy, Action]",[United States of America],300000000,6.9,4500,139.082615,961000000,0
2,206647,Spectre,2015-10-26,148.0,en,"[Action, Adventure, Crime]","[United Kingdom, United States of America]",245000000,6.3,4466,107.376788,880674609,0
3,49026,The Dark Knight Rises,2012-07-16,165.0,en,"[Action, Crime, Drama, Thriller]",[United States of America],250000000,7.6,9106,112.312950,1084939099,0
4,49529,John Carter,2012-03-07,132.0,en,"[Action, Adventure, Science Fiction]",[United States of America],260000000,6.1,2124,43.926995,284139100,0


In [518]:
def get_cast(row):
    
    id1 = str(row['id'])
    response = requests.get('https://api.themoviedb.org/3/movie/'+id1+'/credits?api_key='+SECRET_KEY)
    results = response.json()
    if 'crew' in results.keys():
        dirs = []
        for x in results['crew'][:5]:
            if x['department'] == 'Directing':
                dirs.append(x['name'])
        if len(dirs)>0: return dirs
        else: return 0
    else: return 0

df.head().apply(get_cast, axis=1)


0        [James Cameron]
1       [Gore Verbinski]
2           [Sam Mendes]
3    [Christopher Nolan]
4       [Andrew Stanton]
dtype: object

In [ ]:
def get_cast(row):
    
    id1 = str(row['id'])
    response = requests.get('https://api.themoviedb.org/3/movie/'+id1+'/credits?api_key='+SECRET_KEY)
    results = response.json()
    if 'cast' in results.keys():
        return [(results['cast'][i]['name']) for i in range(3)]
    else: return 0

df['cast'] = df.apply(get_cast, axis=1)

In [30]:



load_dotenv(dotenv_path='.env')
SECRET_KEY = os.getenv("TOKEN")

In [ ]:
id1 = 19995
response = requests.get('https://api.themoviedb.org/3/movie/'+id1+'/external_ids?api_key='+SECRET_KEY)
results = response.json()
results["imdb_id"]

In [515]:

response = requests.get('https://api.themoviedb.org/3/movie/206647/credits?api_key='+SECRET_KEY)
results = response.json()
results.keys()

dict_keys(['id', 'cast', 'crew'])

In [503]:
# direct = results['cast'][i]['name']
direct = [(results['cast'][i]['name']) for i in range(3)]
direct

['Sam Worthington', 'Zoe Saldana', 'Stephen Lang']

In [517]:
direct = results['crew']
lista = []
for x in direct:
    if x['department'] == 'Directing':
        lista.append(x['name'])
        break
direct

[{'credit_id': '5751eed59251416b60000637',
  'department': 'Production',
  'gender': 2,
  'id': 1477203,
  'job': 'Co-Producer',
  'name': 'Luca Marco Paracels',
  'profile_path': None},
 {'credit_id': '52fe4d22c3a368484e1d8d77',
  'department': 'Directing',
  'gender': 2,
  'id': 39,
  'job': 'Director',
  'name': 'Sam Mendes',
  'profile_path': '/5z89X9rB76JDblqMQ52fviwXxAN.jpg'},
 {'credit_id': '52fe4d22c3a368484e1d8d7d',
  'department': 'Production',
  'gender': 1,
  'id': 10666,
  'job': 'Producer',
  'name': 'Barbara Broccoli',
  'profile_path': None},
 {'credit_id': '52fe4d22c3a368484e1d8d83',
  'department': 'Production',
  'gender': 2,
  'id': 69678,
  'job': 'Producer',
  'name': 'Michael G. Wilson',
  'profile_path': '/j9E7oKJMgprlrwkUkk2JXdY5tnJ.jpg'},
 {'credit_id': '52fe4d22c3a368484e1d8d8d',
  'department': 'Writing',
  'gender': 2,
  'id': 932,
  'job': 'Screenplay',
  'name': 'John Logan',
  'profile_path': '/3yVu9Jda97tJlkijaL3wmSABWwo.jpg'},
 {'credit_id': '54805967c

In [17]:
cast = results['cast'][:3]

In [18]:
cast

[{'cast_id': 242,
  'character': 'Jake Sully',
  'credit_id': '5602a8a7c3a3685532001c9a',
  'gender': 2,
  'id': 65731,
  'name': 'Sam Worthington',
  'order': 0,
  'profile_path': '/9XzAE3ZnCnazub4xrSY8YBN7sNq.jpg'},
 {'cast_id': 3,
  'character': 'Princess Neytiri',
  'credit_id': '52fe48009251416c750ac9cb',
  'gender': 1,
  'id': 8691,
  'name': 'Zoe Saldana',
  'order': 1,
  'profile_path': '/ofNrWiA2KDdqiNxFTLp51HcXUlp.jpg'},
 {'cast_id': 4,
  'character': 'Col. Quaritch',
  'credit_id': '52fe48009251416c750ac9cf',
  'gender': 2,
  'id': 32747,
  'name': 'Stephen Lang',
  'order': 2,
  'profile_path': '/tqF6ibURpLvRPlgvLRvjCQqWaa2.jpg'}]